In [55]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [56]:
# Plot candle price
pair ="EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [57]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [58]:
df.describe() #show for numerical values

,volume
count,5000.000000
mean,3969.037600
std,2928.376527
min,125.000000
25%,2091.500000
50%,3270.500000
75%,4989.250000
max,27524.000000


In [59]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [60]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [61]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()



In [62]:
df_ma.dropna(inplace=True)
df_ma.head(9)

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2023-02-01T05:00:00.000000000Z,1.08657,1.08750,1.08654,1.08717,1.086207,1.086244,1.085132,1.086084,1.087661,1.085795
256,2023-02-01T06:00:00.000000000Z,1.08716,1.08769,1.08660,1.08697,1.086306,1.086379,1.085190,1.086091,1.087648,1.085797
257,2023-02-01T07:00:00.000000000Z,1.08697,1.08839,1.08667,1.08801,1.086547,1.086504,1.085284,1.086156,1.087640,1.085828
258,2023-02-01T08:00:00.000000000Z,1.08804,1.08885,1.08698,1.08875,1.086859,1.086694,1.085389,1.086204,1.087640,1.085865
259,2023-02-01T09:00:00.000000000Z,1.08874,1.08886,1.08787,1.08836,1.087189,1.086808,1.085469,1.086231,1.087633,1.085905
260,2023-02-01T10:00:00.000000000Z,1.08810,1.08938,1.08797,1.08923,1.087671,1.086933,1.085596,1.086272,1.087629,1.085943
261,2023-02-01T11:00:00.000000000Z,1.08924,1.08970,1.08887,1.08926,1.088041,1.087110,1.085746,1.086310,1.087626,1.085981
262,2023-02-01T12:00:00.000000000Z,1.08926,1.08980,1.08880,1.08918,1.088366,1.087234,1.085888,1.086351,1.087620,1.086019
263,2023-02-01T13:00:00.000000000Z,1.08918,1.09102,1.08883,1.09040,1.088770,1.087489,1.086105,1.086407,1.087625,1.086065


In [63]:
df_plot = df_ma.iloc[-200:]

In [64]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))


fig.update_layout(width=1000,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()
